Будем использовать реализацию Word2vec в библиотеке **Gensim**, а в качестве предобученных моделей возьмем модели Андрея Кутузова и Лизы Кузьменко с сайта [RusVectōrēs.](https://rusvectores.org/ru/models/). 

In [1]:
!pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 167kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
import gensim
gensim.__version__

'3.8.3'

In [3]:
%load_ext autoreload

from gensim.models import Word2Vec, KeyedVectors

В качестве моделей давайте возьмем 

1) araneum_none_fasttextcbow_300_5_2018 (fasttext) - модель, обученная на интернет-корпусе русского языка


2) ruscorpora_upos_skipgram_300_5_2018 (word2vec) - модель, обученная НКРЯ

## word2vec + fasttext

Существуют несколько форматов, в которых могут храниться модели - .vec и .model 

1) Первый формат считается классическим вариантом модели word2vec. Для загрузки таакой модели надо воспользоваться методом *KeyedVectors.load_word2vec_format*. 
Модель может быть бинарной, для ее загрузки надо передать параметр binary, равный True. 

2) Формат .model - собственный формат gensim. Такую модель надо загружать с помощью метода *KeyedVectors.load*.

 **1) если модель без тэгов**

In [4]:
!wget -c https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz

--2020-10-18 11:57:25--  https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2691248108 (2.5G) [application/x-gzip]
Saving to: ‘araneum_none_fasttextcbow_300_5_2018.tgz’

araneum_none_fastte 100%[===================>]   2.51G  27.8MB/s    in 93s     

2020-10-18 11:58:58 (27.6 MB/s) - ‘araneum_none_fasttextcbow_300_5_2018.tgz’ saved [2691248108/2691248108]



In [5]:
!tar -xzf araneum_none_fasttextcbow_300_5_2018.tgz

In [6]:
# загрузка модели

model_file = 'araneum_none_fasttextcbow_300_5_2018.model'
model = KeyedVectors.load(model_file)

## Получение вектора документа

Отлично, вектора для слов получены. Что с ними делать дальше? 

Есть два подхода (а точнее есть один, а второй мы придумали, потому что с одним жить нельзя).
> Классика - для получения вектора документа нужно взять и усреднить все вектора его слов
 
$$ vec_{doc} = \frac {\sum_{i=0}^{n} vec_i}{len(d)} $$


In [ ]:
import numpy as np

# сделали препроцессинг, получили леммы 
lemmas = ['старинный', 'замок']

# создаем вектор-маску
lemmas_vectors = np.zeros((len(lemmas), model.vector_size))

vec = np.zeros((model.vector_size,))

# если слово есть в модели, берем его вектор
for idx, lemma in enumerate(lemmas):
    if lemma in model:
        lemmas_vectors[idx] = model[lemma]
        
# проверка на случай, если на вход пришел пустой массив
if lemmas_vectors.shape[0] is not 0:
    vec = np.mean(lemmas_vectors, axis=0)


> Эксперимент - представим документ не в виде одного уредненного вектора, а как матрицу векторов входящих в него слов

```
 слово1 |  v1_300
 слово2 |  v2_300
 слово3 |  v3_300
 слово4 |  v4_300
```

> Отлично, теперь каждый документ представлен в виде матрицы векторов своих слов. Но нам надо получить близость матрицы документа в коллекции и матрицы входящего запроса. Как? Умножим две матрицы друг на друга - одна матрица размером d x 300, другая q x 300 - получим попарную близость слов из каждого документа - матрицу размером d x q.


In [ ]:
# возьмем игрушечный пример кейса

text1 = 'турция' 
text2 = 'нужна справка срочно'
query = 'быстрая справка'

In [ ]:
# построим матрицы всех документов

def create_doc_matrix(text):
    lemmas = text.split()
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
    vec = np.zeros((model.vector_size,))

    for idx, lemma in enumerate(lemmas):
        if lemma in model.wv:
            lemmas_vectors[idx] = normalize_vec(model.wv[lemma])
            
    return lemmas_vectors    


text1_m = create_doc_matrix(text1)
text2_m = create_doc_matrix(text2)
query_m = create_doc_matrix(query)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
# размер матрицы как и ожидали
query_m.shape

(2, 300)

In [ ]:
# посмотрим на близость слов первого текста и слов запроса
text1_m.dot(query_m.T)

array([[0.09587915, 0.01183069]])

In [ ]:
# посмотрим на близость слов второго текста и слов запроса
text2_m.dot(query_m.T)

array([[-0.0260624 ,  0.11607588],
       [ 0.01341236,  1.00000011],
       [ 0.22505549,  0.33582122]])

In [ ]:
docs_m = [text2_m, text1_m]

    
def search_1(docs, query, reduce_func=np.max, axis=0):
    sims = []
    for doc in docs:
        sim = doc.dot(query.T)
        sim = reduce_func(sim, axis=axis)
        sims.append(sim.sum())
    print(sims)
    return np.argmax(sims)


r = search_1(docs_m, query_m)

[1.225055597288777, 0.10770983955697251]


## Задание

Реализуйте поиск по нашему стандартному Covid корпусу с помощью модели на Araneum двумя способами:

    1. преобразуйте каждый документ в вектор через усреднение векторов его слов и реализуйте поисковик как 
    обычно через умножение матрицы документов коллекции на вектор запроса 
    2. экспериментальный способ - реализуйте поиск ближайшего документа в коллекции к запросу, преобразовав 
    каждый документ в матрицу (количество слов x размер модели)
    
Посчитайте качество поиска для каждой модели на тех же данных, что и в предыдущем задании. В качестве препроцессинга используйте две версии - с удалением NER и без удаления.  



---


Во время выполнения домашнего задания 3 я сохранила проепроцессиг с NER в отдельные файлы, поэтому в данном домашнем задании я буду использовать результаты предыдущего дз.

#### Подготовка: препроцессинг с NER и без него

In [7]:
import re
import pandas as pd
import numpy as np

In [8]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [9]:
%cd /content/gdrive/My Drive/Вышечка/4 курс/инфопоиск/семинар 4

/content/gdrive/My Drive/Вышечка/4 курс/инфопоиск/семинар 4


In [10]:
answers = pd.read_csv('answers_new_funcs.csv', delimiter='\t')
queries = pd.read_csv("queries_new_funcs.csv", delimiter='\t')

In [11]:
queries = queries.fillna(0)

In [12]:
for_queries = queries[['Номер связки\n', 'Текст вопроса',
                       'preprocess_natasha',	'preprocess_deepmipt']]

In [13]:
for_queries.rename(columns={'Номер связки\n': 'Номер связки',
                            'Текст вопроса': 'Текст вопросов',
                            'preprocess_natasha': 'preprocess_natasha',
                            'preprocess_deepmipt': 'preprocess_deepmipt'},
                   inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [14]:
for_answers = answers[['Номер связки', 'Текст вопросов',
                       'preprocess_natasha',	'preprocess_deepmipt']]

In [15]:
from sklearn.model_selection import train_test_split

RANDOM_STATE = 35

train_df, test = train_test_split(for_queries, test_size=0.3,
                               random_state=RANDOM_STATE)

In [16]:
train = pd.concat([for_answers, train_df])

In [17]:
!pip install --quiet pymorphy2[fast]

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 8.2MB 6.2MB/s 
     |████████████████████████████████| 378kB 45.1MB/s 


In [18]:
from pymorphy2 import MorphAnalyzer


morph = MorphAnalyzer()

In [19]:
def preprocessing(data):
    data2 = []
    cnt = 0
    for v in data:
        strng = ''
        line = "".join(l for l in str(v) if l not in (
            '.', ',', '?', '!', ':', ';', '—', '--', '<', '>', '"', '(', ')'))
        for word in line.split():
            word = re.sub(r'\ufeff', '', word)
            wrd = morph.parse(word)[0]
            strng += wrd.normal_form + ' '
        data2.append(strng)
    return data2

In [20]:
from tqdm.auto import tqdm

In [21]:
lemm_text = preprocessing(tqdm(train['Текст вопросов']))
train['вопросы_препроцессинговые'] = lemm_text

In [22]:
lemm_text2 = preprocessing(tqdm(train['preprocess_deepmipt']))
train['deeppavlov'] = lemm_text2

In [23]:
lemm_text3 = preprocessing(tqdm(train['preprocess_natasha']))
train['natasha'] = lemm_text3

In [24]:
train.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,deeppavlov,natasha
0,57.0,У ребенка в школе продлили каникулы. Могу ли я...,У ребенка в школе продлили каникулы. Могу ли я...,У ребенка в школе продлили каникулы . Могу ли ...,у ребёнок в школа продлить каникулы мочь ли я ...,у ребёнок в школа продлить каникулы мочь ли я ...,у ребёнок в школа продлить каникулы мочь ли я ...
1,78.0,Где сделать вакцинацию от коронавируса?\nСущес...,Где сделать вакцинацию от коронавируса?\nСущес...,Где сделать вакцинацию от коронавируса ? Сущес...,где сделать вакцинация от коронавирус существо...,где сделать вакцинация от коронавирус существо...,где сделать вакцинация от коронавирус существо...
2,326.0,Сколько стоит сделать вакцину от гриппа?\nМожн...,Сколько стоит сделать вакцину от гриппа?\nМожн...,Сколько стоит сделать вакцину от гриппа ? Можн...,сколько стоить сделать вакцина от грипп можно ...,сколько стоить сделать вакцина от грипп можно ...,сколько стоить сделать вакцина от грипп можно ...
3,327.0,Могу я отказаться от вакцинации?\nВ каких случ...,Могу я отказаться от вакцинации?\nВ каких случ...,Могу я отказаться от вакцинации ? В каких случ...,мочь я отказаться от вакцинация в какой случай...,мочь я отказаться от вакцинация в какой случай...,мочь я отказаться от вакцинация в какой случай...
4,328.0,Безопасна ли вакцинация?\nОпасна ли вакцинация...,Безопасна ли вакцинация?\nОпасна ли вакцинация...,Безопасна ли вакцинация ? Опасна ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...,безопасный ли вакцинация опасный ли вакцинация...


In [25]:
lemm_text4 = preprocessing(tqdm(test['Текст вопросов']))
test['вопросы_препроцессинговые'] = lemm_text4

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
lemm_text5 = preprocessing(tqdm(test['preprocess_deepmipt']))
test['deeppavlov'] = lemm_text5

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
lemm_text6 = preprocessing(tqdm(test['preprocess_natasha']))
test['natasha'] = lemm_text6

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
test.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,deeppavlov,natasha
1333,308.0,"В сентябре, хочу поехать на личном транспорте ...","В сентябре, хочу поехать на личном транспорте ...","В сентябре , хочу поехать на личном транспорте...",в сентябрь хотеть поехать на личный транспорт ...,в сентябрь хотеть поехать на личный транспорт ...,в сентябрь хотеть поехать на личный транспорт ...
1986,308.0,ДОбрый день!\n\nя прилетел в воскресенье рано ...,ДОбрый день!\n\nя прилетел в воскресенье рано ...,ДОбрый день ! \n \n я прилетел в воскресенье р...,добрый день я прилететь в воскресение рано утр...,добрый день я прилететь в воскресение рано утр...,добрый день я прилететь в воскресение рано утр...
2282,37.0,Добрый день! Нужна ли справка об отсутствии Ко...,Добрый день! Нужна ли справка об отсутствии п...,Добрый день ! Нужна ли справка об отсутствии п...,добрый день нужный ли справка о отсутствие ков...,добрый день нужный ли справка о отсутствие при...,добрый день нужный ли справка о отсутствие при...
810,5.0,Здравствуйте. Я работаю врачем в поликлинике. ...,Здравствуйте. Я работаю врачем в поликлинике. ...,Здравствуйте . Я работаю врачем в поликлинике ...,здравствуйте я работать вракать в поликлиника ...,здравствуйте я работать вракать в поликлиника ...,здравствуйте я работать вракать в поликлиника ...
2147,1.0,"\nЗдравствуйте!\nСкажите, пожалуйста, граждани...","\nЗдравствуйте!\nСкажите, пожалуйста, граждани...","Здравствуйте ! Скажите , пожалуйста , граждани...",здравствуйте сказать пожалуйста гражданин само...,здравствуйте сказать пожалуйста гражданин само...,здравствуйте сказать пожалуйста гражданин само...


#### Матрицы

In [29]:
def normalize_vec(v):
     return v / np.sqrt(np.sum(v ** 2))

In [30]:
def create_doc_matrix(text):
    lemmas = text.split()
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
    vec = np.zeros((model.vector_size,))

    for idx, lemma in enumerate(lemmas):
        if lemma in model.wv:
            lemmas_vectors[idx] = normalize_vec(model.wv[lemma])
            
    return lemmas_vectors 

##### Препроцессинг без NER

In [31]:
vec_train = []
for index, row in tqdm(train.iterrows()):
    text1 = create_doc_matrix(row['вопросы_препроцессинговые'])
    vec_train.append(text1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [32]:
def search(docs, query_m, reduce_func=np.max, axis=0):
    sims = []
    query = create_doc_matrix(query_m)
    for doc in docs:
        sim = doc.dot(query.T)
        sim = reduce_func(sim, axis=axis)
        sims.append(sim.sum())
    return sims

In [33]:
n_conn = train['Номер связки'].tolist()

In [34]:
def ranging_simple(text):
    cnt = np.argmax(text)
    return n_conn[cnt]

In [35]:
for_no_prep = []
for index, row in tqdm(test.iterrows()):
    fin = search(vec_train, row['вопросы_препроцессинговые'])
    for_no_prep.append(ranging_simple(fin))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [36]:
test['no_prep_res'] = for_no_prep

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
test.tail()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,deeppavlov,natasha,no_prep_res
479,57.0,"Здравствуйте! Я работаю в больнице, нас закрыв...","Здравствуйте! Я работаю в больнице, нас закрыв...","Здравствуйте ! Я работаю в больнице , нас закр...",здравствуйте я работать в больница мы закрыват...,здравствуйте я работать в больница мы закрыват...,здравствуйте я работать в больница мы закрыват...,308.0
1545,257.0,"Добрый день, с 07.08.2020 года почувствовал не...","Добрый день, с 07.08.2020 года почувствовал не...","Добрый день , с 07 . 08 . 2020 года почувствов...",добрый день с 07082020 год почувствовать недом...,добрый день с 07 08 2020 год почувствовать нед...,добрый день с 07082020 год почувствовать недом...,308.0
1473,308.0,"Добрый день! \nСкажите пожалуйста, может ли пр...","Добрый день! \nСкажите пожалуйста, может ли пр...","Добрый день ! Скажите пожалуйста , может ли пр...",добрый день сказать пожалуйста мочь ли прилете...,добрый день сказать пожалуйста мочь ли прилете...,добрый день сказать пожалуйста мочь ли прилете...,308.0
278,12.0,Добрый день! \n \nВыдано постано...,Добрый день! \n \nВыдано постано...,Добрый день ! \n \n Выдано постановление на 14...,добрый день выдать постановление на 14 день як...,добрый день выдать постановление на 14 день як...,добрый день выдать постановление на 14 день як...,308.0
1348,326.0,Как сделать прививку от гриппа?\n,Как сделать прививку от гриппа?\n,Как сделать прививку от гриппа ?,как сделать прививка от грипп,как сделать прививка от грипп,как сделать прививка от грипп,327.0


###### Оценивание качества

In [38]:
from sklearn.metrics import accuracy_score

In [39]:
accuracy_score(test['Номер связки'], test['no_prep_res'])

0.47246376811594204

##### Препроцессинг c NER

In [54]:
vec_train_ner = []
for index, row in tqdm(train.iterrows()):
    text2 = create_doc_matrix(row['natasha'])
    vec_train_ner.append(text2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [55]:
for_ner = []
for index, row in tqdm(test.iterrows()):
    fin = search(vec_train_ner, row['natasha'])
    for_ner.append(ranging_simple(fin))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [56]:
test['ner_res'] = for_ner

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [57]:
test.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,deeppavlov,natasha,no_prep_res,ner_res
1333,308.0,"В сентябре, хочу поехать на личном транспорте ...","В сентябре, хочу поехать на личном транспорте ...","В сентябре , хочу поехать на личном транспорте...",в сентябрь хотеть поехать на личный транспорт ...,в сентябрь хотеть поехать на личный транспорт ...,в сентябрь хотеть поехать на личный транспорт ...,308.0,308.0
1986,308.0,ДОбрый день!\n\nя прилетел в воскресенье рано ...,ДОбрый день!\n\nя прилетел в воскресенье рано ...,ДОбрый день ! \n \n я прилетел в воскресенье р...,добрый день я прилететь в воскресение рано утр...,добрый день я прилететь в воскресение рано утр...,добрый день я прилететь в воскресение рано утр...,308.0,308.0
2282,37.0,Добрый день! Нужна ли справка об отсутствии Ко...,Добрый день! Нужна ли справка об отсутствии п...,Добрый день ! Нужна ли справка об отсутствии п...,добрый день нужный ли справка о отсутствие ков...,добрый день нужный ли справка о отсутствие при...,добрый день нужный ли справка о отсутствие при...,308.0,308.0
810,5.0,Здравствуйте. Я работаю врачем в поликлинике. ...,Здравствуйте. Я работаю врачем в поликлинике. ...,Здравствуйте . Я работаю врачем в поликлинике ...,здравствуйте я работать вракать в поликлиника ...,здравствуйте я работать вракать в поликлиника ...,здравствуйте я работать вракать в поликлиника ...,308.0,12.0
2147,1.0,"\nЗдравствуйте!\nСкажите, пожалуйста, граждани...","\nЗдравствуйте!\nСкажите, пожалуйста, граждани...","Здравствуйте ! Скажите , пожалуйста , граждани...",здравствуйте сказать пожалуйста гражданин само...,здравствуйте сказать пожалуйста гражданин само...,здравствуйте сказать пожалуйста гражданин само...,308.0,308.0


###### Оценивание качества

In [58]:
accuracy_score(test['Номер связки'], test['ner_res'])

0.46956521739130436

#### Вектора

##### Препроцессинг без NER

In [59]:
def doc_vector(text):

    # сделали препроцессинг, получили леммы 
    lemmas = text.split()

    # создаем вектор-маску
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))

    vec = np.zeros((model.vector_size,))

    # если слово есть в модели, берем его вектор
    for idx, lemma in enumerate(lemmas):
        if lemma in model:
            lemmas_vectors[idx] = model[lemma]
            
    # проверка на случай, если на вход пришел пустой массив
    if lemmas_vectors.shape[0] is not 0:
        vec = np.mean(lemmas_vectors, axis=0)

    return vec

In [60]:
for_wth_pr = []

for index, row in tqdm(train.iterrows()):
    fin = doc_vector(row['вопросы_препроцессинговые'])
    for_wth_pr.append(normalize_vec(fin))

In [61]:
matr = np.array(for_wth_pr)

In [62]:
matr

array([[ 0.01112431, -0.01080169, -0.01695967, ..., -0.07221471,
        -0.0817516 ,  0.04181375],
       [ 0.0763685 ,  0.00948331, -0.08365379, ..., -0.02470702,
        -0.07871651,  0.0702822 ],
       [-0.0201066 ,  0.00323688, -0.15226098, ..., -0.00850061,
        -0.05224903,  0.00250972],
       ...,
       [-0.00034379, -0.0115637 , -0.04060583, ..., -0.09947407,
        -0.02659727,  0.01715896],
       [ 0.00725594,  0.05643359, -0.04997718, ..., -0.03429967,
        -0.08505712, -0.09479618],
       [-0.03352605,  0.07929043, -0.10247447, ..., -0.03918518,
        -0.03724929,  0.11981027]])

In [63]:
def sim_vectors(matr, query):
  doc = doc_vector(query)
  sim = matr.dot(doc.T)
  return sim

In [64]:
for_pre_test = []
for index, row in tqdm(test.iterrows()):
    fin = sim_vectors(matr, row['вопросы_препроцессинговые'])
    for_pre_test.append(ranging_simple(fin))

In [66]:
test['vec_sim_no_prep'] = for_pre_test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [67]:
test.head()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,deeppavlov,natasha,no_prep_res,ner_res,vec_sim_no_prep
1333,308.0,"В сентябре, хочу поехать на личном транспорте ...","В сентябре, хочу поехать на личном транспорте ...","В сентябре , хочу поехать на личном транспорте...",в сентябрь хотеть поехать на личный транспорт ...,в сентябрь хотеть поехать на личный транспорт ...,в сентябрь хотеть поехать на личный транспорт ...,308.0,308.0,308.0
1986,308.0,ДОбрый день!\n\nя прилетел в воскресенье рано ...,ДОбрый день!\n\nя прилетел в воскресенье рано ...,ДОбрый день ! \n \n я прилетел в воскресенье р...,добрый день я прилететь в воскресение рано утр...,добрый день я прилететь в воскресение рано утр...,добрый день я прилететь в воскресение рано утр...,308.0,308.0,308.0
2282,37.0,Добрый день! Нужна ли справка об отсутствии Ко...,Добрый день! Нужна ли справка об отсутствии п...,Добрый день ! Нужна ли справка об отсутствии п...,добрый день нужный ли справка о отсутствие ков...,добрый день нужный ли справка о отсутствие при...,добрый день нужный ли справка о отсутствие при...,308.0,308.0,37.0
810,5.0,Здравствуйте. Я работаю врачем в поликлинике. ...,Здравствуйте. Я работаю врачем в поликлинике. ...,Здравствуйте . Я работаю врачем в поликлинике ...,здравствуйте я работать вракать в поликлиника ...,здравствуйте я работать вракать в поликлиника ...,здравствуйте я работать вракать в поликлиника ...,308.0,12.0,308.0
2147,1.0,"\nЗдравствуйте!\nСкажите, пожалуйста, граждани...","\nЗдравствуйте!\nСкажите, пожалуйста, граждани...","Здравствуйте ! Скажите , пожалуйста , граждани...",здравствуйте сказать пожалуйста гражданин само...,здравствуйте сказать пожалуйста гражданин само...,здравствуйте сказать пожалуйста гражданин само...,308.0,308.0,1.0


###### Оценивание качества

In [68]:
accuracy_score(test['Номер связки'], test['vec_sim_no_prep'])

0.5405797101449276

##### Препроцессинг c NER

###### Оценивание качества

In [69]:
for_vec_ner = []

for index, row in tqdm(train.iterrows()):
    fin = doc_vector(row['natasha'])
    for_vec_ner.append(normalize_vec(fin))

In [70]:
matr_2 = np.array(for_vec_ner)

In [71]:
matr_2.shape

(1652, 300)

In [72]:
for_ner_test = []

for index, row in tqdm(test.iterrows()):
    fin = sim_vectors(matr_2, row['natasha'])
    for_ner_test.append(ranging_simple(fin))

In [73]:
test['vec_sim_ner'] = for_ner_test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [74]:
test.tail()

,Номер связки,Текст вопросов,preprocess_natasha,preprocess_deepmipt,вопросы_препроцессинговые,deeppavlov,natasha,no_prep_res,ner_res,vec_sim_no_prep,vec_sim_ner
479,57.0,"Здравствуйте! Я работаю в больнице, нас закрыв...","Здравствуйте! Я работаю в больнице, нас закрыв...","Здравствуйте ! Я работаю в больнице , нас закр...",здравствуйте я работать в больница мы закрыват...,здравствуйте я работать в больница мы закрыват...,здравствуйте я работать в больница мы закрыват...,308.0,308.0,57.0,1.0
1545,257.0,"Добрый день, с 07.08.2020 года почувствовал не...","Добрый день, с 07.08.2020 года почувствовал не...","Добрый день , с 07 . 08 . 2020 года почувствов...",добрый день с 07082020 год почувствовать недом...,добрый день с 07 08 2020 год почувствовать нед...,добрый день с 07082020 год почувствовать недом...,308.0,308.0,1.0,1.0
1473,308.0,"Добрый день! \nСкажите пожалуйста, может ли пр...","Добрый день! \nСкажите пожалуйста, может ли пр...","Добрый день ! Скажите пожалуйста , может ли пр...",добрый день сказать пожалуйста мочь ли прилете...,добрый день сказать пожалуйста мочь ли прилете...,добрый день сказать пожалуйста мочь ли прилете...,308.0,308.0,308.0,308.0
278,12.0,Добрый день! \n \nВыдано постано...,Добрый день! \n \nВыдано постано...,Добрый день ! \n \n Выдано постановление на 14...,добрый день выдать постановление на 14 день як...,добрый день выдать постановление на 14 день як...,добрый день выдать постановление на 14 день як...,308.0,308.0,12.0,12.0
1348,326.0,Как сделать прививку от гриппа?\n,Как сделать прививку от гриппа?\n,Как сделать прививку от гриппа ?,как сделать прививка от грипп,как сделать прививка от грипп,как сделать прививка от грипп,327.0,327.0,326.0,326.0


In [75]:
accuracy_score(test['Номер связки'], test['vec_sim_ner'])

0.5144927536231884